In [13]:
import numpy as np
from numpy import linalg as LA
from scipy.sparse.linalg import LinearOperator, eigsh
from timeit import default_timer as timer

from ncon import ncon

Indexing convention:

    -For H, is with free indices pointing downwards and numeration from left to right in increaring order
    -For each coupling h (rank 4 tensor), downwards -> upwards left-to-right so DOWN:(1,2) UP:(3,4)

In [19]:
import numpy as np


def doApplyHam(psiIn: np.ndarray,
               hloc: np.ndarray,
               N: int,
               usePBC: bool):
  """
  Applies local Hamiltonian, given as sum of nearest neighbor terms, to
  an input quantum state.
  Args:
    psiIn: vector of length d**N describing the quantum state.
    hloc: array of ndim=4 describing the nearest neighbor coupling.
    N: the number of lattice sites.
    usePBC: sets whether to include periodic boundary term.
  Returns:
    np.ndarray: state psi after application of the Hamiltonian.
  """
  d = hloc.shape[0]
  psiOut = np.zeros(psiIn.size)
  for k in range(N - 1):
    # apply local Hamiltonian terms to sites [k,k+1]
    psiOut += np.tensordot(hloc.reshape(d**2, d**2),
                           psiIn.reshape(d**k, d**2, d**(N - 2 - k)),
                           axes=[[1], [1]]).transpose(1, 0, 2).reshape(d**N)

  if usePBC:
    # apply periodic term
    psiOut += np.tensordot(hloc.reshape(d, d, d, d),
                           psiIn.reshape(d, d**(N - 2), d),
                           axes=[[2, 3], [2, 0]]
                           ).transpose(1, 2, 0).reshape(d**N)

  return psiOut

In [24]:
model = 'ising'
Nsites = 18
usePCB = False
numval = 1

d = 2
sX = np.array([[0, 1.0],[1.0, 0]])
sY = np.array([[0, -1.0j],[1.0j, 0]])
sZ = np.array([[1.0, 0],[0, -1.0]])
sI = np.array([[1.0, 0],[0, 1.0]])

if model == 'XX':
    hloc = (np.real(np.kron(sX,sX) + np.kron(sY,sY))).reshape(d,d,d,d)
elif model =='ising':
    hloc = (0.5*np.kron(sZ,sI) + 0.5*np.kron(sI,sZ) - np.kron(sX,sX)).reshape(d,d,d,d)

def doApplyHamClosed(psiIn):
    return doApplyHam(psiIn, hloc, Nsites, usePCB)

H = LinearOperator((2**Nsites, 2**Nsites), matvec=doApplyHamClosed)

start_time = timer()
Energy, psi = eigsh(H, k= numval, which='SA')
end_time = timer()

print('NumSites: %d, Time elapsed: %1.2f, Energy: %e'%(Nsites, end_time - start_time, Energy[0]))

NumSites: 18, Time elapsed: 12.39, Energy: -2.177107e+01
